In [1]:
# ------------------------------------------
# CELDA 1: Importar bibliotecas necesarias
# ------------------------------------------

import sqlite3  # Importamos la biblioteca para conectarnos a una base de datos SQLite
import csv      # Importamos csv para poder leer archivos CSV
import os       # Importamos os para manejar rutas y archivos en el sistema operativo

# Definimos las rutas de archivos en variables para facilidad de mantenimiento
# Ajusta estas rutas si tienes una estructura de carpetas diferente.
data_folder = "../data"        # Carpeta donde se ubican los archivos CSV
db_folder   = "../db"          # Carpeta donde se guardará la base de datos
db_name     = "production.db"  # Nombre del archivo de base de datos
print(f'Bibliotecas instaladas')

Bibliotecas instaladas


In [2]:
# ------------------------------------------
# CELDA 2: Crear/conectar la base de datos
# ------------------------------------------

# Verificamos si la carpeta db existe. Si no, la creamos.
if not os.path.exists(db_folder):
    os.makedirs(db_folder)  # Creamos la carpeta db si no existe

# Construimos la ruta completa a la base de datos
db_path = os.path.join(db_folder, db_name)

# Creamos la conexión a la base de datos SQLite
# Si el archivo production.db no existe, se creará automáticamente.
conn = sqlite3.connect(db_path)

# Creamos un cursor que nos permitirá ejecutar comandos SQL
cursor = conn.cursor()

print(f"Base de datos creada/conectada en: {db_path}")


Base de datos creada/conectada en: ../db\production.db


In [3]:
# ------------------------------------------
# CELDA 3: Crear tablas (orders y products)
# ------------------------------------------

# Eliminamos las tablas si existen (solo para reiniciar el ejemplo cada vez)
cursor.execute("DROP TABLE IF EXISTS orders;")   # Borramos la tabla 'orders' si existía
cursor.execute("DROP TABLE IF EXISTS products;") # Borramos la tabla 'products' si existía

# Creamos la tabla 'orders' con las columnas apropiadas
create_orders_table = """
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    product_id INTEGER,
    quantity INTEGER,
    order_date TEXT,
    status TEXT
);
"""

# Creamos la tabla 'products' con las columnas apropiadas
create_products_table = """
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    price REAL,
    category TEXT
);
"""

# Ejecutamos las sentencias de creación
cursor.execute(create_orders_table)
cursor.execute(create_products_table)

# Confirmamos los cambios
conn.commit()

print("Tablas 'orders' y 'products' creadas exitosamente.")


Tablas 'orders' y 'products' creadas exitosamente.


In [4]:
# ------------------------------------------
# CELDA 4: Cargar datos CSV a las tablas
# ------------------------------------------

# Definimos la ruta de cada archivo CSV
orders_csv_path = os.path.join(data_folder, "orders.csv")
products_csv_path = os.path.join(data_folder, "products.csv")

# Cargamos datos en la tabla 'orders'
with open(orders_csv_path, mode='r', encoding='utf-8-sig') as f:  # Abrimos el archivo CSV de orders
    reader = csv.DictReader(f)  # DictReader para acceder a las columnas por nombre
    for row in reader:
        # Extraemos cada valor (conversión de tipos si corresponde)
        order_id = int(row['order_id'])            # Convertimos a int
        product_id = int(row['product_id'])        # Convertimos a int
        quantity = int(row['quantity'])            # Convertimos a int
        order_date = row['order_date']             # Cadena de texto
        status = row['status']                     # Cadena de texto
        
        # Preparamos la sentencia INSERT con placeholders
        insert_order_query = """
        INSERT INTO orders (order_id, product_id, quantity, order_date, status)
        VALUES (?, ?, ?, ?, ?);
        """
        
        # Ejecutamos la inserción con la tupla de valores
        cursor.execute(insert_order_query, (order_id, product_id, quantity, order_date, status))

# Cargamos datos en la tabla 'products'
with open(products_csv_path, mode='r', encoding='utf-8-sig') as f:
    reader = csv.DictReader(f)
    for row in reader:
        product_id = int(row['product_id'])
        product_name = row['product_name']
        price = float(row['price'])
        category = row['category']
        
        insert_product_query = """
        INSERT INTO products (product_id, product_name, price, category)
        VALUES (?, ?, ?, ?);
        """
        
        cursor.execute(insert_product_query, (product_id, product_name, price, category))

# Confirmamos los cambios
conn.commit()

print("Datos cargados correctamente desde CSV a las tablas.")


Datos cargados correctamente desde CSV a las tablas.
